# Laboratorium 1 - analiza koszykowa

## Przygotowanie

 * pobierz i wypakuj dataset: https://kaggle.com/datasets/rashikrahmanpritom/groceries-dataset-for-market-basket-analysismba?resource=download&select=basket.csv
   * alternatywnie, pobierz plik `basket.csv` z Teamsów
 * [opcjonalnie] Utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab1`
 * zainstaluj potrzebne biblioteki:
 `pip install more-itertools`

## Część 1. - przygotowanie danych

In [2]:
# importujemy wszystkie potrzebne pakiety

from more_itertools import powerset

In [3]:
# definiujemy stale

PATH = './basket.csv'
EPSILON = 0.001
K = 4

In [4]:
# wczytujemy dane o koszykach

def read_baskets(path: str) -> list[tuple[str]]:
    with open(path) as f:
        raw = f.read()
    baskets = [set([y.lower() for y in x.split(',') if y]) for x in raw.split('\n')[1:] if x]
    return baskets

def unique_products(baskets: list[tuple[str]]) -> list[str]:
    products = set()
    for basket in baskets:
        products.update(basket)
    return sorted(list(products))

baskets = read_baskets(PATH)
products = unique_products(baskets)

## Część 2. - obliczanie wskaźników

In [5]:
# obliczamy strukture danych (np. slownik albo graf) przechowujaca wszystkie interesujace wartosci `support`
from itertools import combinations
from collections import Counter
import math

# apriori
def get_supports(baskets: list[set], all_products: list[str], epsilon: float):
    """
    Level-wise Apriori: returns supports as {tuple(sorted(items)): support_fraction}.
    """
    n = len(baskets)
    if n == 0:
        return {}
    
    min_count = max(1, math.ceil(epsilon * n))
    supports = {}

    # 1-Item-Sets
    cnt1 = Counter()
    for b in baskets:
        for item in b:
            cnt1[item] += 1
    Lk = []
    for item, c in cnt1.items():
        if c >= min_count:
            key = (item,)
            supports[tuple(sorted(key))] = c / n
            Lk.append(key)

    k = 2

    # 2 and more Item-Sets
    while Lk and k <= K:
        prev = [tuple(sorted(x)) for x in Lk]
        prev.sort()
        candidates = set()
        for a, b in combinations(prev, 2):
            # join only if first k-2 items equal (same core)
            if a[:-1] == b[:-1]:
                cand = tuple(sorted(set(a) | set(b)))
                if len(cand) == k:
                    candidates.add(cand)

        if not candidates:
            break

        # prune candidates whose any (k-1)-subset is not frequent
        prev_set = set(prev)
        pruned = []
        for cand in candidates:
            ok = True
            for subset in combinations(cand, k-1):
                if tuple(sorted(subset)) not in prev_set:
                    ok = False
                    break
            if ok:
                pruned.append(cand)
        if not pruned:
            break

        # count pruned candidates by scanning baskets once
        counts = Counter()
        pruned_set_keys = set(pruned)
    
        for b in baskets:
            if len(b) < k:
                continue
            # generate k-subsets of the basket and count those present in candidate list
            for comb in combinations(b, k):
                key = tuple(sorted(comb))
                if key in pruned_set_keys:
                    counts[key] += 1

        Lk = []
        for cand, c in counts.items():
            if c >= min_count:
                supports[tuple(sorted(cand))] = c / n
                Lk.append(cand)

        k += 1

    return supports
    
supports = get_supports(baskets, products, EPSILON)
supports

{('salty snack',): 0.018779656485998796,
 ('pastry',): 0.0517275947336764,
 ('whole milk',): 0.15792287642852368,
 ('yogurt',): 0.08587850030074183,
 ('semi-finished bread',): 0.009490075519615051,
 ('sausage',): 0.06034886052262247,
 ('pickled vegetables',): 0.008955423377664907,
 ('soda',): 0.09710619528169484,
 ('canned beer',): 0.04691572545612511,
 ('misc. beverages',): 0.01577223818752924,
 ('hygiene articles',): 0.013700461137472432,
 ('rolls/buns',): 0.11000467820624206,
 ('whipped/sour cream',): 0.043707812604424245,
 ('frankfurter',): 0.037759807525228894,
 ('curd',): 0.03368308494285905,
 ('white bread',): 0.023992514870012697,
 ('beef',): 0.03395041101383412,
 ('butter',): 0.03522020985096572,
 ('other vegetables',): 0.12210118291786407,
 ('frozen vegetables',): 0.028002405934638777,
 ('sugar',): 0.01771035220209851,
 ('tropical fruit',): 0.0677671589921807,
 ('specialty chocolate',): 0.015972732740760543,
 ('butter milk',): 0.017576689166610975,
 ('frozen meals',): 0.01677

In [ ]:
# definiujemy funkcje obliczajace support, confidence i lift

def support(supports, products: tuple[str]) -> float:
    key = tuple(sorted(products))
    if key in supports:
        return supports[key]
    else:
        return 0.0


def confidence(supports, prior_products: tuple[str], following_products: tuple[str]) -> float:
    if (support(supports, prior_products) == 0):
        return 0.0
    else:
        return support(supports, prior_products + following_products) / support(supports, prior_products)
    
def lift(supports, prior_products: tuple[str], following_products: tuple[str]) -> float:
    if (support(supports, prior_products) == 0 or support(supports, following_products) == 0):
        return 0.0
    else:
        return support(supports, prior_products + following_products) / (support(supports, prior_products) * support(supports, following_products))
    

0.0014702933903628951


In [ ]:
print(support(supports, {'whole milk', 'rolls/buns'}))
print(confidence(supports, {'whole milk', 'rolls/buns'}, {'yogurt'}))
print(lift(supports, {'whole milk', 'rolls/buns'}, {'yogurt'}))

## Część 3. - generowanie rekomendacji

In [37]:
# wyznaczamy liste potencjalnych rekomendacji
# rekomendowane artykuly powinny miec lift > 1 i jak najwyzszy confidence

def generate_basic_candidates(basket: tuple[str], products: list[str], supports) -> list[tuple[str, tuple[str], float, float]]:
    candidates = []
    basket_set = set(basket)

    for i in range(1, len(basket_set) + 1):
        for subbasket in combinations(basket_set, i):
            subbasket = tuple(sorted(subbasket))
            
            for product in products:
                if product not in basket_set:
                    current_lift = lift(supports, subbasket, (product,))
                    if current_lift > 1:
                        current_confidence = confidence(supports, subbasket, (product,))
                        candidates.append((product, subbasket, current_confidence, current_lift))
    
    candidates.sort(key=lambda x: (x[2], x[3]), reverse=True)
    
    return candidates

In [38]:
# zaproponuj drugi, bardziej zaawansowany algorytm, np.:
# - jesli produkt X wystepuje w liscie kandydatow kilkukrotnie, oblicz srednia lub iloczyn confidence
# - posortuj kandydatow po iloczynie configence i lift

def generate_advanced_candidates(basket: tuple[str], products: list[str], supports) -> list[tuple[str, tuple[str], float, float]]:
    basic_recs = generate_basic_candidates(basket, products, supports)
    
    if not basic_recs:
        return []

    aggregated_rules = {}
    for item, sub, conf, lift in basic_recs:
        if item not in aggregated_rules:
            aggregated_rules[item] = {
                'best_rule': (item, sub, conf, lift),
                'confidences': [conf], 
                'lifts': [lift]
            }
        else:
            aggregated_rules[item]['confidences'].append(conf)
            aggregated_rules[item]['lifts'].append(lift)
    
            if conf > aggregated_rules[item]['best_rule'][2]:
                aggregated_rules[item]['best_rule'] = (item, sub, conf, lift)

    final_candidates = []
    for item, data in aggregated_rules.items():
        avg_conf = sum(data['confidences']) / len(data['confidences'])
        avg_lift = sum(data['lifts']) / len(data['lifts'])
        
        sort_score = avg_conf * avg_lift
        final_candidates.append((data['best_rule'], sort_score))

    final_candidates.sort(key=lambda x: x[1], reverse=True)
    
    return [rule for rule, score in final_candidates]

In [39]:
print(baskets[1])

print("basic candidates:")
print(generate_basic_candidates(baskets[1], products, supports)[:5])

print("advanced candidates:")
print(generate_advanced_candidates(baskets[1], products, supports)[:5])

{'yogurt', 'whole milk', 'semi-finished bread', 'sausage'}
basic candidates:
[('rolls/buns', ('sausage', 'whole milk'), 0.12686567164179105, 1.1532752398396837), ('rolls/buns', ('whole milk', 'yogurt'), 0.11976047904191618, 1.0886853267947703), ('soda', ('sausage', 'whole milk'), 0.11940298507462688, 1.2296124333596987), ('soda', ('sausage',), 0.09856035437430785, 1.0149749363405152), ('bottled beer', ('sausage',), 0.05537098560354374, 1.2220000849348451)]
advanced candidates:
[('rolls/buns', ('sausage', 'whole milk'), 0.12686567164179105, 1.1532752398396837), ('soda', ('sausage', 'whole milk'), 0.11940298507462688, 1.2296124333596987), ('curd', ('sausage',), 0.04872646733111849, 1.4466153386419167), ('bottled beer', ('sausage',), 0.05537098560354374, 1.2220000849348451), ('pastry', ('sausage',), 0.05315614617940199, 1.0276168156103256)]


In [42]:
print(baskets[33])

print("basic candidates:")
print(generate_basic_candidates(baskets[33], products, supports))

print("advanced candidates:")
print(generate_advanced_candidates(baskets[33], products, supports)[:5])

{'yogurt', 'tropical fruit', 'domestic eggs', 'white wine', 'root vegetables', 'photo/film', 'soda'}
basic candidates:
[('sausage', ('yogurt',), 0.0669260700389105, 1.1089864739670185), ('sausage', ('soda',), 0.06125258086717137, 1.0149749363405152), ('citrus fruit', ('yogurt',), 0.053696498054474705, 1.0106423904265471), ('shopping bags', ('root vegetables',), 0.04803073967339097, 1.0093875810856026), ('newspapers', ('domestic eggs',), 0.04144144144144145, 1.0654437943097737), ('coffee', ('domestic eggs',), 0.03783783783783784, 1.1969716016227643), ('frankfurter', ('domestic eggs',), 0.03783783783783784, 1.0020664912700312), ('frozen vegetables', ('root vegetables',), 0.030739673390970224, 1.0977511526231203), ('white bread', ('domestic eggs',), 0.02702702702702703, 1.126477452382745), ('uht-milk', ('tropical fruit',), 0.022682445759368838, 1.0606169871794873), ('specialty chocolate', ('tropical fruit',), 0.019723865877712035, 1.2348460465615276), ('beverages', ('soda',), 0.0192704748